In [169]:
import pandas as pd
import numpy as np
import os
import cv2
import re
import unicodedata as ud


In [6]:
## read data in

In [176]:
def split_coor_basin(basin_file):
    ''' split coordinate for each basin row data
    '''
    basin = pd.read_excel(basin_file)
    basin.set_index('Unnamed: 0',inplace=True)
    basin_row_con = {} ## data container
#     basin_row_2d = [] ## basin row container
    #iterate rows
    for i in range(1,len(basin)):
        basin_row_list = [] # row data container
        basin_row = basin.iloc[i]
        for j in range(len(basin_row)):
            basin_row_pari = []
            if ',' in str(basin_row[j]) and basin_row[j]:
                basin_row_col_split = str(basin_row[j]).split(',') #split data
                if re.match(r'-*\d',basin_row_col_split[0]) and re.match(r'-*\d',basin_row_col_split[1]): ##data is not null and valid
                    basin_row_pari.append(basin_row_col_split[0]) #saving long
                    basin_row_pari.append(basin_row_col_split[1]) #saving lat
            basin_row_list.append(basin_row_pari) #saving pair to basin_row
        basin_row_con[basin_row.name] = basin_row_list ## append list to 2d row container
#         basin_row_con[basin_row.name] = basin_row_2d
    return basin_row_con

In [177]:
split_basin_data = split_coor_basin('basin.xlsx')

In [179]:
basin_test_df = pd.DataFrame.from_dict(split_basin_data['Faroe - Shetland Escarpment'])

In [191]:
def gen_field_df(split_basin_data):
    '''generate dataframe for each field with such column,basin,lat,and long
    
    '''
    basin_tuple = {}
    basin_dfs = []
    for key in split_basin_data.keys(): ##iterate through the libaries
        basin_df = pd.DataFrame.from_dict(split_basin_data[key]) ## gen dataframe
        basin_df = basin_df[~basin_df.isnull()]
        basin_df.dropna(inplace=True)
        basin_df['basin'] = key
        basin_dfs.append(basin_df) ## append dataframe to list
        
    
    basin_df_con = pd.concat(basin_dfs) ##concata together
#     basin_df_con = basin_df_con[~basin_df_con.isnull()] ##find null value
#     basin_df_con.dropna(inplace=True)
    basin_df_con.columns = ['long','lat','basin']
    basin_df_con['long'] = basin_df_con['long'].map(float)
    basin_df_con['lat'] = basin_df_con['lat'].map(float)
    #set index
    basin_df_con.set_index('basin',inplace=True)
    
    return basin_df_con

In [192]:
basin_dfs = gen_field_df(split_basin_data)

In [215]:
def get_basin_tuple(basin_dfs):
    '''convert basin row data to list of tuple
    '''
    basins_dict = {}
    basins = basin_dfs.index.unique()
    for basin in basins:
         
        basin_ready_tuple = basin_dfs[basin_dfs.index == basin] ##basin index equal to one of element
        basin_tuples = [tuple(x) for x in basin_ready_tuple.values] ## convert data frame to tuple
        basins_dict[basin] = basin_tuples
        
    return basins_dict
        
        
        

In [216]:
basin_tuple = get_basin_tuple(basin_dfs)

In [314]:
def check_inside_poly(field_coor,basin_tuple,basin):
    '''field coordinate and one basin_tuple
    Args:
        field coordinate,(1,2),tuple
        basin tuple,[(1,2),(3,4).......],list
        basin, basin name,string
    Return:
        inside or not and basin name,(True,'Basin_name')
    '''
    inside_rect = False #inside flag
    crossing_point = 0
    max_long = max([x[0] for x in basin_tuple]) #max long
    min_long = min([x[0] for x in basin_tuple]) #min long
    max_lat = max([x[1] for x in basin_tuple]) #max lat
    min_lat = min([x[1] for x in basin_tuple]) #min lat
    if (min_long<=field_coor[0] and max_long>=field_coor[1] 
        or min_lat<field_coor[1] and max_lat>field_coor[1]): # point inside the rectangele
        for i in range(len(basin_tuple)): #iterate the poly points
            slope = (basin_tuple[i+1][1] - basin_tuple[i][1])/ (basin_tuple[i+1][0] - basin_tuple[i][0])
            if (field_coor[0] - basin_tuple[i][1])/(field_coor[1]-basin[i][0]) <= slope:
                crossing_point += 1
        if crossing_point%2 == 0:
            inside_rect = True
            
    return inside_rect,basin
    
    
    
    

In [316]:
def determin_range(field_file,basin_tuple):
    '''project field lat and long to correspoding basin name
    
    Args:
        field,dataframe with storing field coordinates
        basin,basin coordinates
    Returns:
        field-basin, corresponding dataframe
        
    '''
    ## read in field data
    field = pd.read_excel(field_file)
#     basin = pd.read_excel('basin.xlsx')
#     field_basin = {}
    field_arr = field[['field','long','lat']]
    field_arr.set_index('field',inplace=True)
    field_arr['basin'] = None
    for i in range(len(field_arr)):
        field_row = field_arr.iloc[i] ## get field coord
        field_coor = tuple(field_row.to_list()) ## get field coordinate tuple
        ## for each point searching inside basins
        for key in basin_tuple.keys(): ## for one basin
            ##check for each basin coorinates
            is_inside,basin = check_field_basin(field_coor,basin_tuple[key],key) ##match field to basin
            if is_inside: ## if point inside this basin
                field_arr['basin'] = basin_name ## save basin name to field row
    return field_arr

In [162]:
basins_range = get_lat_long_range(split_basin_data)

In [240]:
field_basin = get_basin('field.xlsx',basins_range)

D:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for la

In [243]:
field_basin.to_excel('field_basin.xlsx')

In [286]:
class Point:
    lng = None
    lat = None
 
    def __init__(self, lng, lat):
        self.lng = lng
        self.lat = lat
        
def get_polygon_bounds(points):
    length = len(points)
    top = down = left = right = points[0]
    for i in range(1, length):
        if points[i].lng > top.lng:
            top = points[i]
        elif points[i].lng < down.lng:
            down = points[i]
        else:
            pass
        if points[i].lat > right.lat:
            right = points[i]
        elif points[i].lat < left.lat:
            left = points[i]
        else:
            pass
    top_left = Point(top.lng, left.lat)
    top_right = Point(top.lng, right.lat)
    down_right = Point(down.lng, right.lat)
    down_left = Point(down.lng, left.lat)
    return [top_left, top_right, down_right, down_left]

def is_point_in_rect(point, polygon_bounds):
    top_left = polygon_bounds[0]
    top_right = polygon_bounds[1]
    down_right = polygon_bounds[2]
    down_left = polygon_bounds[3]
    return (down_left.lng <= point.lng <= top_right.lng
            and top_left.lat <= point.lat <= down_right.lat)

def is_point_in_polygon(point, points):
    polygon_bounds = get_polygon_bounds(points)
    if not is_point_in_rect(point, polygon_bounds):
        return False
    length = len(points)
    point_start = points[0]
    flag = False
    for i in range(1, length):
        point_end = points[i]
        # 点与多边形顶点重合
        if (point.lng == point_start.lng and point.lat == point_start.lat) or (
                point.lng == point_end.lng and point.lat == point_end.lat):
            return True
        # 判断线段两端点是否在射线两侧
        if (point_end.lat < point.lat <= point_start.lat) or (
                point_end.lat >= point.lat > point_start.lat):
            # 线段上与射线 Y 坐标相同的点的 X 坐标
            if point_end.lat == point_start.lat:
                x = (point_start.lng + point_end.lng) / 2
            else:
                x = point_end.lng - (point_end.lat - point.lat) * (
                        point_end.lng - point_start.lng) / (
                        point_end.lat - point_start.lat)
            # 点在多边形的边上
            if x == point.lng:
                return True
            # 射线穿过多边形的边界
            if x > point.lng:
                flag = not flag
            else:
                pass
        else:
            pass
 
        point_start = point_end
    return flag

def test(input_lng=116.732617, input_lat=39.722676):
    # polyline 是多个坐标点，形如
    # ['116.732617,39.722676', '116.732617,39.722676', '116.732617,39.722676',
    # '116.732617,39.722676', '116.732617,39.722676']
    polyline = []
 
    points = []
    for line in polyline:
        if line:
            try:
                lng, lat = line.split(',')
                points.append(Point(float(lng), float(lat)))
            except ValueError:
                pass
    if points:
        is_point_in_polygon(Point(float(input_lng), float(input_lat)), points)